In [1]:
import torch
from torch import nn
import numpy
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs

In [2]:
NUM_CLASSES=4
NUM_FEATURES=2
RANDOM_SEED=42

In [23]:
X_blob,y_blob = make_blobs(n_samples=1000,n_features=NUM_FEATURES,centers=NUM_CLASSES,cluster_std=1.5,random_state=RANDOM_SEED)
X_blob = torch.from_numpy(X_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor)
X_train,X_test,y_train,y_test = train_test_split(X_blob,y_blob,test_size=0.2,random_state=RANDOM_SEED)

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [25]:
X_train,y_train = X_train.to(device),y_train.to(device)
X_test,y_test = X_test.to(device),y_test.to(device)

In [26]:
class Model_1(nn.Module):
  def __init__(self,input_features,output_features,hidden_units):
    super().__init__()
    self.linear_1 = nn.Linear(in_features=input_features,out_features=hidden_units)
    self.linear_2 = nn.Linear(in_features=hidden_units,out_features=hidden_units)
    self.linear_3 = nn.Linear(in_features=hidden_units,out_features=output_features)
    self.relu = nn.ReLU()
  def forward(self,x):
    return self.linear_3(self.relu(self.linear_2(self.relu(self.linear_1(x)))))

In [27]:
model_1 = Model_1(input_features=NUM_FEATURES,output_features=NUM_CLASSES,hidden_units=8).to(device)

In [29]:
EPOCHS=100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_1.parameters(),lr=0.01)

In [18]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [19]:
from torchmetrics import Accuracy

In [20]:
test_accuracy = Accuracy(task='multiclass',num_classes=NUM_CLASSES).to(device)

In [30]:
for epoch in range(EPOCHS):
  model_1.train()

  y_logits = model_1(X_train)
  y_pred = torch.softmax(y_logits,dim=1).argmax(dim=1)

  loss = loss_fn(y_logits,y_train)
  optimizer.zero_grad()

  loss.backward()
  optimizer.step()

  model_1.eval()
  with torch.inference_mode():
    test_logits = model_1(X_test)
    test_pred = torch.softmax(test_logits,dim=1).argmax(dim=1)
    test_loss = loss_fn(test_logits,y_test)
    test_acc = test_accuracy(test_pred,y_test)

  if epoch%10 == 0:
    print(f'Epoch:{epoch} Loss:{loss:.5f} TestAccuracy:{test_acc:.2f}')

Epoch:0 Loss:1.21625 TestAccuracy:0.54
Epoch:10 Loss:0.60143 TestAccuracy:0.95
Epoch:20 Loss:0.18566 TestAccuracy:0.99
Epoch:30 Loss:0.06242 TestAccuracy:0.99
Epoch:40 Loss:0.03645 TestAccuracy:1.00
Epoch:50 Loss:0.02909 TestAccuracy:1.00
Epoch:60 Loss:0.02633 TestAccuracy:1.00
Epoch:70 Loss:0.02498 TestAccuracy:1.00
Epoch:80 Loss:0.02414 TestAccuracy:1.00
Epoch:90 Loss:0.02351 TestAccuracy:1.00
